# **INFO5731 In-class Exercise 2**

The purpose of this exercise is to understand users' information needs, and then collect data from different sources for analysis by implementing web scraping using Python.

**Expectations**:
*   Students are expected to complete the exercise during lecture period to meet the active participation criteria of the course.
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

**Total points**: 40

**Deadline**: This in-class exercise is due at the end of the day tomorrow, at 11:59 PM.

**Late submissions will have a penalty of 10% of the marks for each day of late submission. , and no requests will be answered. Manage your time accordingly.**


## Question 1 (10 Points)
Describe an interesting research question (or practical question or something innovative) you have in mind, what kind of data should be collected to answer the question(s)? Specify the amount of data needed for analysis. Provide detailed steps for collecting and saving the data.

# write your answer here
Reasearch Question- Analyzing book pricing across different generes in a website
Data Required- To analyze the book pricing fo different generes we need to collect data like title of the book, price of the book and genere of the
book
Amount of data required- For simple analysis we need a data set of 1000 books with their title, price and genere. This data can help us to identify
trends over pricing and genere.
Steps to collect and save the data-
1. Identifying website to scrape data- We have to choose a website from which we can perform scraping and it should have data of price and genere of the book
2. Setting up environent- We have to install python libraries such as BeautifulSoup and pandas
3. Web Scraping- Understanding the HTML of web page and performing scraping using soup for parsing the webpage and requests to connect with the webpage
4. Saving data- Convert the collected data using pandas dataframe and save it as a csv file and you can download the csv using google colab's
   files.download()

## Question 2 (10 Points)
Write Python code to collect a dataset of 1000 samples related to the question discussed in Question 1.

In [ ]:
# write your answer here
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import files
# List for storing book data
book_data_list= []
# URL for book store and page no is the url is dynamic
web_url = "http://books.toscrape.com/catalogue/page-{}.html"
for page_number in range(1, 51):  #website has 50 pages of books
    url = web_url.format(page_number)
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Connected to the page no {page_number}!")
    else:
        print(f"Connection failed to the page {page_number}. Status_Code:{response.status_code}")
        break
    # Using soap for parsing the web page
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all('article', class_='product_pod')
    # Looping through the book details
    for book in books:
        title = book.h3.a['title']
        price = book.find('p', class_='price_color').text
        availability = book.find('p', class_='instock availability').text.strip()

        # Using dictonary to store the book details
        book_data = {
            'title': title,
            'price': price,
            'availability': availability
        }
        book_data_list.append(book_data)
    # Stopping at 1000 records
    if len(book_data) >= 1000:
        break
data = book_data_list[:1000]
df = pd.DataFrame(data)
df.to_csv('book_data.csv', index=False)
files.download('book_data.csv')
print("Web scraping completed")

Successfully connected to the page no 1!
Successfully connected to the page no 2!
Successfully connected to the page no 3!
Successfully connected to the page no 4!
Successfully connected to the page no 5!
Successfully connected to the page no 6!
Successfully connected to the page no 7!
Successfully connected to the page no 8!
Successfully connected to the page no 9!
Successfully connected to the page no 10!
Successfully connected to the page no 11!
Successfully connected to the page no 12!
Successfully connected to the page no 13!
Successfully connected to the page no 14!
Successfully connected to the page no 15!
Successfully connected to the page no 16!
Successfully connected to the page no 17!
Successfully connected to the page no 18!
Successfully connected to the page no 19!
Successfully connected to the page no 20!
Successfully connected to the page no 21!
Successfully connected to the page no 22!
Successfully connected to the page no 23!
Successfully connected to the page no 24!
S

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Web scraping completed


## Question 3 (10 Points)
Write Python code to collect 1000 articles from Google Scholar (https://scholar.google.com/), Microsoft Academic (https://academic.microsoft.com/home), or CiteSeerX (https://citeseerx.ist.psu.edu/index), or Semantic Scholar (https://www.semanticscholar.org/), or ACM Digital Libraries (https://dl.acm.org/) with the keyword "XYZ". The articles should be published in the last 10 years (2014-2024).

The following information from the article needs to be collected:

(1) Title of the article

(2) Venue/journal/conference being published

(3) Year

(4) Authors

(5) Abstract

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import files
# Username and password for serp api of oxy labs
USERNAME = "Deepa_EaHaq"
PASSWORD = "Deepa20020202_"

def get_html_for_page(url):
    payload = {
        "url": url,
        "source": "google",
    }
    response = requests.post(
        "https://realtime.oxylabs.io/v1/queries",
        auth=(USERNAME, PASSWORD),
        json=payload,
    )
    response.raise_for_status()
    return response.json()["results"][0]["content"]

def parse_data_from_article(article):
    title_elem = article.find("h3", {"class": "gs_rt"})
    title = title_elem.get_text()

    title_anchor_elem = article.select("a")[0]
    url = title_anchor_elem["href"]
    # Extracting authors and venues
    authors_and_venue = article.find("div", {"class": "gs_a"}).get_text()
    authors = "Unknown"
    venue_info = "Unknown"
    year = "Unknown"
    try:
        parts = authors_and_venue.split(" - ")
        authors = parts[0]
        venue_info = parts[1]

        # Extracting year
        year = venue_info.split(",")[-1].strip()
    except IndexError:
        pass

    # Extracting abstract data
    abstract_elem = article.find("div", {"class": "gs_rs"})
    abstract = abstract_elem.get_text(strip=True) if abstract_elem else "No abstract available"

    return {
        "title": title,
        "authors": authors,
        "venue": venue_info,
        "year": year,
        "abstract": abstract,
        "url": url,
    }

def get_url_for_page(url, page_index):
    return url + f"&start={page_index}"

def get_data_from_page(url):
    html = get_html_for_page(url)
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.find_all("div", {"class": "gs_ri"})
    return [parse_data_from_article(article) for article in articles]

def scrape_data_for_topic(topic, num_pages=10):
    base_url = f"https://scholar.google.com/scholar?q={topic}&hl=en&as_sdt=0,5"
    data = []
    page_index = 0

    while len(data) < 1000:
        page_url = get_url_for_page(base_url, page_index)
        entries = get_data_from_page(page_url)
        if not entries:  # breaking out of loop when no records are found
            break
        data.extend(entries)
        page_index += 10
        if page_index >= num_pages * 10:
            break

    return data

# List of topics to scrape data from google scholar
topics = [
    "cloud computing",
    "devops",
    "artificial intelligence",
    "machine learning",
    "natural language processing",
    "open ai",
    "web development",
    "data science",
    "data analytics",
    "big data",
    "frontend development",
    "backend development",
    "full stack development",
    "mobile app development",
    "iot",
    "cybersecurity",
    "blockchain",
    "python",
    "java",
    "javascript",
]

# Intializing data list
all_data = []
for topic in topics:
    print(f"Scraping data for topic: {topic}")
    topic_data = scrape_data_for_topic(topic)
    all_data.extend(topic_data)
    if len(all_data) >= 1000:
        break

# Limitting to 1000 records
all_data = all_data[:1000]
df = pd.DataFrame(all_data)
excel_filename = 'research_topics_data.xlsx'
df.to_excel(excel_filename, index=False)
files.download(excel_filename)
print("Data scraping completed")


Scraping data for topic: cloud computing
Scraping data for topic: devops
Scraping data for topic: artificial intelligence
Scraping data for topic: machine learning
Scraping data for topic: natural language processing
Scraping data for topic: open ai
Scraping data for topic: web development
Scraping data for topic: data science
Scraping data for topic: data analytics
Scraping data for topic: big data
Scraping data for topic: frontend development
Scraping data for topic: backend development
Scraping data for topic: full stack development
Scraping data for topic: mobile app development
Scraping data for topic: iot


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data scraping completed


## Question 4A (10 Points)
Develop Python code to collect data from social media platforms like Reddit, Instagram, Twitter (formerly known as X), Facebook, or any other. Use hashtags, keywords, usernames, or user IDs to gather the data.



Ensure that the collected data has more than four columns.


In [ ]:
import random
from ntscraper import Nitter
import pandas as pd
from google.colab import files
# Intializing scrapper
scraper = Nitter(log_level=1, skip_instance_check=False)
tweets = scraper.get_tweets("MrBeast", mode="user", number=1000)
# Intializing tweet data
tweet_data = {
    'URL': [],
    'Tweet Text': [],
    'Username': [],
    'Likes Count': [],
    'Quotes Count': [],
    'Retweets Count': [],
    'Comments Count': []
}
# Selecting random tweets
tweet_limit = random.randint(30, 50)
shuffled_tweets = random.sample(tweets['tweets'], len(tweets['tweets']))
empty_fields_count = 0
# Iterating throught shuffled tweets
for i, tweet in enumerate(shuffled_tweets):
    if i >= tweet_limit:
        break
    if not tweet['link'] or not tweet['text'] or not tweet['user']['name']:
        empty_fields_count += 1
        continue
    tweet_data['URL'].append(tweet['link'])
    tweet_data['Tweet Text'].append(tweet['text'])
    tweet_data['Username'].append(tweet['user']['name'])
    tweet_data['Likes Count'].append(tweet['stats']['likes'] if 'likes' in tweet['stats'] else 0)
    tweet_data['Quotes Count'].append(tweet['stats']['quotes'] if 'quotes' in tweet['stats'] else 0)
    tweet_data['Retweets Count'].append(tweet['stats']['retweets'] if 'retweets' in tweet['stats'] else 0)
    tweet_data['Comments Count'].append(tweet['stats']['comments'] if 'comments' in tweet['stats'] else 0)
# Logging
print(f"Skipped {empty_fields_count} tweets because of missing data.")
# Creating data frame to save to excel
tweet_df = pd.DataFrame(tweet_data)
output_filename = "MrBeast_tweets_data_randomized.xlsx"
tweet_df.to_excel(output_filename, index=False)
files.download(output_filename)
print("Scraping completed")


Testing instances: 100%|██████████| 16/16 [00:15<00:00,  1.01it/s]
INFO:root:No instance specified, using random instance https://nitter.privacydev.net
INFO:root:Current stats for MrBeast: 17 tweets, 0 threads...
INFO:root:Current stats for MrBeast: 33 tweets, 0 threads...
INFO:root:Current stats for MrBeast: 48 tweets, 0 threads...
INFO:root:Current stats for MrBeast: 64 tweets, 0 threads...
INFO:root:Current stats for MrBeast: 81 tweets, 0 threads...


Skipped 0 tweets because of missing data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Scraping completed


## Question 4B (10 Points)
If you encounter challenges with Question-4 web scraping using Python, employ any online tools such as ParseHub or Octoparse for data extraction. Introduce the selected tool, outline the steps for web scraping, and showcase the final output in formats like CSV or Excel.



Upload a document (Word or PDF File) in any shared storage (preferably UNT OneDrive) and add the publicly accessible link in the below code cell.

Please only choose one option for question 4. If you do both options, we will grade only the first one

# Mandatory Question

**Important: Reflective Feedback on Web Scraping and Data Collection**



Please share your thoughts and feedback on the web scraping and data collection exercises you have completed in this assignment. Consider the following points in your response:



Learning Experience: Describe your overall learning experience in working on web scraping tasks. What were the key concepts or techniques you found most beneficial in understanding the process of extracting data from various online sources?



Challenges Encountered: Were there specific difficulties in collecting data from certain websites, and how did you overcome them? If you opted for the non-coding option, share your experience with the chosen tool.



Relevance to Your Field of Study: How might the ability to gather and analyze data from online sources enhance your work or research?

**(no grading of your submission if this question is left unanswered)**

In [ ]:
'''
The task is challenging, some of the key concepts i have explored is BeautifulSoup, request library which i have used frequently. I found
BeautifulSoup quite benficial to extract data and the challenges i encountered while working on this assignment is rate limiting and blocking.
I have used oxylabs serpscraper api to overcome these challenges and this assignment is highly relavent to my field of study for data collection and
data science and also for some reasearch purposes.
'''